## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für Messwerte zu berechnen.   
Zudem wird die Gieränderung in rad bestimmt. Die Ergebnisse können zurück in die Merkmal CSV Datei geschrieben werden.

In [173]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [174]:
featuresDf = pandas.read_csv("merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,Messwerte,StartZeitfenster,StartBewegungsdaten,StartBelichtung,Belichtungszeit
count,1.484500e+04,14845.000000,14845.000000,14845.000000,14845.000000,1.484500e+04,1.484500e+04,1.484500e+04,1.484500e+04
mean,1.521126e+12,53.616914,10.188200,14.596723,19.438464,3.221171e+13,3.221165e+13,3.221174e+13,2.109333e+07
std,1.313290e+06,0.007137,0.027214,3.372945,0.563356,1.287856e+12,1.287857e+12,1.287854e+12,7.078451e+06
min,1.521124e+12,53.606853,10.137816,5.040000,6.000000,2.999506e+13,2.999499e+13,2.999509e+13,6.150027e+06
25%,1.521125e+12,53.612995,10.167364,12.348000,19.000000,3.108906e+13,3.108899e+13,3.108909e+13,2.000368e+07
50%,1.521126e+12,53.614210,10.186600,14.760000,19.000000,3.219892e+13,3.219886e+13,3.219895e+13,2.000368e+07
75%,1.521127e+12,53.620880,10.211833,17.136000,20.000000,3.336325e+13,3.336319e+13,3.336328e+13,2.999632e+07
max,1.521128e+12,53.632652,10.229659,24.768002,22.000000,3.445658e+13,3.445653e+13,3.445660e+13,3.287369e+07


In [175]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14845 entries, 0 to 14844
Data columns (total 16 columns):
Zeitstempel            14845 non-null int64
Breitengrad            14845 non-null float64
Laengengrad            14845 non-null float64
Geschwindigkeit        14845 non-null float64
AccelerometerX         14845 non-null object
AccelerometerY         14845 non-null object
AccelerometerZ         14845 non-null object
Azimuth                14845 non-null object
Nick                   14845 non-null object
Roll                   14845 non-null object
SensorZeitstempel      14845 non-null object
Messwerte              14845 non-null int64
StartZeitfenster       14845 non-null int64
StartBewegungsdaten    14845 non-null int64
StartBelichtung        14845 non-null int64
Belichtungszeit        14845 non-null int64
dtypes: float64(3), int64(6), object(7)
memory usage: 1.8+ MB


In [176]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf.AccelerometerX.head()

0    0.66929 0.81137 0.75647 0.22221 -0.14389 -0.13...
1    0.2285 0.13695 0.37016 0.08172 -0.05709 0.3834...
2    -2.72057 3.72297 3.37689 -1.28232 -0.90316 0.4...
3    -0.06289 0.54739 0.6219 -1.35643 -0.54872 0.28...
4    -0.5752 -1.41006 -1.3118 -0.58965 -1.32969 0.6...
Name: AccelerometerX, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
AccelerometerX       non-null object   
AccelerometerY       non-null object   
AccelerometerZ       non-null object   
Azimuth              non-null object   
Nick                 non-null object   
Roll                 non-null object   
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [177]:
accXList = featuresDf.AccelerometerX.str.split(" ").tolist()

In [178]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf.AccelerometerX.str.split(" ")).values

array([[ ['0.66929', '0.81137', '0.75647', '0.22221', '-0.14389', '-0.13032', '-0.34931', '-0.26401', '-0.349', '-0.41699', '-0.27218', '-0.38619', '-0.29352', '-0.46454', '-0.47877', '-0.3829', '-0.13767', '-0.11002', '0.01936', '0.27609']],
       [ ['0.2285', '0.13695', '0.37016', '0.08172', '-0.05709', '0.38348', '-0.27536', '-0.6339', '0.21317', '-0.05919', '-0.44563', '-0.78543', '-0.52098', '-1.10619', '-0.19531', '-0.7384', '-0.60594', '-0.54593', '0.00775', '-0.26951']],
       [ ['-2.72057', '3.72297', '3.37689', '-1.28232', '-0.90316', '0.47276', '0.60817', '1.13021', '1.93092', '0.62547', '-2.50279', '-0.54644', '0.81943', '-0.67742', '-0.06682', '-0.23722', '0.45391', '1.206', '1.94558', '2.88967']],
       ..., 
       [ ['1.49666', '-0.16642', '-1.12912', '-0.53554', '0.16916', '0.02807', '-0.94289', '-0.72366', '-0.0273', '0.62171', '0.54334', '-0.27017', '-0.00162', '0.74952', '0.21655', '-0.33241', '-0.77158', '-0.73985', '-0.0709']],
       [ ['0.79005', '0.203', '0.

In [179]:
accXList[1] # zeige Liste

['0.2285',
 '0.13695',
 '0.37016',
 '0.08172',
 '-0.05709',
 '0.38348',
 '-0.27536',
 '-0.6339',
 '0.21317',
 '-0.05919',
 '-0.44563',
 '-0.78543',
 '-0.52098',
 '-1.10619',
 '-0.19531',
 '-0.7384',
 '-0.60594',
 '-0.54593',
 '0.00775',
 '-0.26951']

In [180]:
np.array(accXList[1]).dtype

dtype('<U8')

In [181]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(np.array(accXList[0]).astype(float)).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())

In [182]:
accXDf = pandas.concat(accList, ignore_index=True)

In [183]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,0.66929,0.81137,0.75647,0.22221,-0.14389,-0.13032,-0.34931,-0.26401,-0.34900,-0.41699,-0.27218,-0.38619,-0.29352,-0.46454,-0.47877,-0.38290,-0.13767,-0.11002,0.01936,0.27609,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,-0.05709,0.38348,-0.27536,-0.63390,0.21317,-0.05919,-0.44563,-0.78543,-0.52098,-1.10619,-0.19531,-0.73840,-0.60594,-0.54593,0.00775,-0.26951,NaN,NaN
2,-2.72057,3.72297,3.37689,-1.28232,-0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,-2.50279,-0.54644,0.81943,-0.67742,-0.06682,-0.23722,0.45391,1.20600,1.94558,2.88967,NaN,NaN
3,-0.06289,0.54739,0.62190,-1.35643,-0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,1.86127,NaN
4,-0.57520,-1.41006,-1.31180,-0.58965,-1.32969,0.65253,1.15038,-0.27477,-0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,-0.67322,-1.87156,-1.45116,0.73922,NaN,NaN


In [184]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
print(accXDf.head(2))
accXDf.T.mean().head(2)

        0        1        2        3        4        5        6        7   \
0  0.66929  0.81137  0.75647  0.22221 -0.14389 -0.13032 -0.34931 -0.26401   
1  0.22850  0.13695  0.37016  0.08172 -0.05709  0.38348 -0.27536 -0.63390   

        8        9        10       11       12       13       14      15  \
0 -0.34900 -0.41699 -0.27218 -0.38619 -0.29352 -0.46454 -0.47877 -0.3829   
1  0.21317 -0.05919 -0.44563 -0.78543 -0.52098 -1.10619 -0.19531 -0.7384   

        16       17       18       19  20  21  
0 -0.13767 -0.11002  0.01936  0.27609 NaN NaN  
1 -0.60594 -0.54593  0.00775 -0.26951 NaN NaN  


0   -0.071226
1   -0.240857
dtype: float64

In [185]:
accXDf.shape

(14845, 22)

In [186]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf.Messwerte[featuresDf.Messwerte == 22]

1472     22
7171     22
7820     22
9598     22
12467    22
14556    22
Name: Messwerte, dtype: int64

In [187]:
# ausführen wenn alle Daten die gleiche Ahnzahl an Messwerten haben sollen. 
# Dadurch werden Reihen gelöscht!
unwantedRows = featuresDf.Messwerte[featuresDf.Messwerte != 20]
featuresDf.drop(featuresDf.index, axis=0)

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,AccelerometerX,AccelerometerY,AccelerometerZ,Azimuth,Nick,Roll,SensorZeitstempel,Messwerte,StartZeitfenster,StartBewegungsdaten,StartBelichtung,Belichtungszeit


In [188]:
accYList = featuresDf.AccelerometerY.str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [189]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14845 entries, 0 to 14844
Data columns (total 22 columns):
0     14845 non-null float64
1     14845 non-null float64
2     14845 non-null float64
3     14845 non-null float64
4     14845 non-null float64
5     14845 non-null float64
6     14844 non-null float64
7     14844 non-null float64
8     14844 non-null float64
9     14843 non-null float64
10    14843 non-null float64
11    14843 non-null float64
12    14843 non-null float64
13    14842 non-null float64
14    14842 non-null float64
15    14842 non-null float64
16    14841 non-null float64
17    14836 non-null float64
18    14642 non-null float64
19    6605 non-null float64
20    134 non-null float64
21    6 non-null float64
dtypes: float64(22)
memory usage: 2.5 MB


In [190]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,0.58300,0.48170,0.38533,-1.20873,-0.46135,-0.75219,-0.57112,0.38583,0.09411,0.19785,0.35745,0.10205,-0.28613,-0.33618,-0.34560,-0.04666,-0.77285,-0.71025,-0.27709,0.65171,NaN,NaN
1,0.28650,-0.61359,-0.29171,0.80856,0.43231,0.85147,-0.02369,-0.04963,0.45060,0.63627,-0.67088,-0.50608,-0.20570,0.15719,1.13704,0.89429,-0.09672,0.36697,0.69194,-0.04407,NaN,NaN
2,-0.48539,-4.54084,-5.10369,-3.63862,-0.93426,0.06467,1.72190,4.53401,3.90300,2.21832,-1.88754,-0.39149,0.49890,-0.82675,-0.69207,-0.01738,0.66028,0.34432,0.06090,-0.57954,NaN,NaN
3,0.26624,-1.24271,-2.05148,-3.17350,-2.00252,-0.17702,0.70112,1.87864,1.05851,1.12260,-0.55762,-0.66064,1.32553,1.22895,-0.07416,-0.02870,2.09158,7.03625,8.37177,4.75138,-1.77646,NaN
4,-0.74329,-4.71651,-2.70064,3.55490,2.53744,-2.88872,-2.51021,-2.20739,-3.19096,-1.89392,1.50345,0.94225,1.64249,1.26800,0.79985,-0.72388,-1.46786,-2.07837,-1.67804,2.68746,NaN,NaN


In [191]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf.AccelerometerZ.str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [192]:
# Konvertiere Nick Daten zu DataFrame
measuredDataList = featuresDf.Nick.str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [193]:
# Konvertiere Roll Daten zu DataFrame
measuredDataList = featuresDf.Roll.str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [194]:
measuredDataList = featuresDf.Azimuth.str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [195]:
accXMean = accXDf.T.mean()
accXMean.head(2)

0   -0.071226
1   -0.240857
dtype: float64

In [196]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['MittelX'] = accXMean
featuresDf['MittelY'] = accYDf.T.mean()
featuresDf['MittelZ'] = accZDf.T.mean()
featuresDf['MittelNick'] = nickDf.T.mean()
featuresDf['MittelRoll'] = rollDf.T.mean()

In [197]:
featuresDf.MittelX.head(2)

0   -0.071226
1   -0.240857
Name: MittelX, dtype: float64

In [198]:
featuresDf.columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartZeitfenster',
       'StartBewegungsdaten', 'StartBelichtung', 'Belichtungszeit', 'MittelX',
       'MittelY', 'MittelZ', 'MittelNick', 'MittelRoll'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [199]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [200]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [201]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [202]:
featuresDf['AzimuthAenderung'] = calculateAngelChangeAzimuth(azimuthDf)

In [203]:
# Berechne Varianz
featuresDf['VarianzX'] = calcVariance(featuresDf.MittelX,accXDf)
featuresDf['VarianzY'] = calcVariance(featuresDf.MittelY,accYDf)
featuresDf['VarianzZ'] = calcVariance(featuresDf.MittelZ,accZDf)
featuresDf['VarianzNick'] = calcVariance(featuresDf.MittelNick,nickDf)
featuresDf['VarianzRoll'] = calcVariance(featuresDf.MittelRoll,rollDf)

In [204]:
featuresDf.VarianzX.head()

0    0.15801
1    0.16816
2    2.87086
3    0.81137
4    0.84476
Name: VarianzX, dtype: float64

In [205]:
calculateStandardDeviation(featuresDf.VarianzX)

array([ 0.3975 ,  0.41007,  1.69436, ...,  0.63688,  0.2118 ,  0.14629])

In [206]:
# Berechne Standardabweichung
featuresDf['AbweichungX'] = calculateStandardDeviation(featuresDf.VarianzX)
featuresDf['AbweichungY'] = calculateStandardDeviation(featuresDf.VarianzY)
featuresDf['AbweichungZ'] = calculateStandardDeviation(featuresDf.VarianzZ)
featuresDf['AbweichungNick'] = calculateStandardDeviation(featuresDf.VarianzNick)
featuresDf['AbweichungRoll'] = calculateStandardDeviation(featuresDf.VarianzRoll)

In [207]:
featuresDf.AbweichungX.head()

0    0.39750
1    0.41007
2    1.69436
3    0.90076
4    0.91911
Name: AbweichungX, dtype: float64

In [208]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14845 entries, 0 to 14844
Data columns (total 22 columns):
0     14845 non-null float64
1     14845 non-null float64
2     14845 non-null float64
3     14845 non-null float64
4     14845 non-null float64
5     14845 non-null float64
6     14844 non-null float64
7     14844 non-null float64
8     14844 non-null float64
9     14843 non-null float64
10    14843 non-null float64
11    14843 non-null float64
12    14843 non-null float64
13    14842 non-null float64
14    14842 non-null float64
15    14842 non-null float64
16    14841 non-null float64
17    14836 non-null float64
18    14642 non-null float64
19    6605 non-null float64
20    134 non-null float64
21    6 non-null float64
dtypes: float64(22)
memory usage: 2.5 MB


In [209]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,0.66929,0.81137,0.75647,0.22221,-0.14389,-0.13032,-0.34931,-0.26401,-0.34900,-0.41699,-0.27218,-0.38619,-0.29352,-0.46454,-0.47877,-0.38290,-0.13767,-0.11002,0.01936,0.27609,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,-0.05709,0.38348,-0.27536,-0.63390,0.21317,-0.05919,-0.44563,-0.78543,-0.52098,-1.10619,-0.19531,-0.73840,-0.60594,-0.54593,0.00775,-0.26951,NaN,NaN
2,-2.72057,3.72297,3.37689,-1.28232,-0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,-2.50279,-0.54644,0.81943,-0.67742,-0.06682,-0.23722,0.45391,1.20600,1.94558,2.88967,NaN,NaN
3,-0.06289,0.54739,0.62190,-1.35643,-0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,1.86127,NaN
4,-0.57520,-1.41006,-1.31180,-0.58965,-1.32969,0.65253,1.15038,-0.27477,-0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,-0.67322,-1.87156,-1.45116,0.73922,NaN,NaN


In [210]:
featuresDf.MittelX.head()

0   -0.071226
1   -0.240857
2    0.512262
3    0.977270
4   -0.202993
Name: MittelX, dtype: float64

In [211]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel', inplace=True)
featuresDf.to_csv('../merkmale.csv')

In [212]:
pandas.read_csv("../merkmale.csv").columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartZeitfenster',
       'StartBewegungsdaten', 'StartBelichtung', 'Belichtungszeit', 'MittelX',
       'MittelY', 'MittelZ', 'MittelNick', 'MittelRoll', 'AzimuthAenderung',
       'VarianzX', 'VarianzY', 'VarianzZ', 'VarianzNick', 'VarianzRoll',
       'AbweichungX', 'AbweichungY', 'AbweichungZ', 'AbweichungNick',
       'AbweichungRoll'],
      dtype='object')

In [213]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.041s

OK
